# Dense Optical Flow with OpenCV

In [2]:
import cv2
import numpy as np

In [8]:
print(F'cv2 version = {cv2.__version__}')
# ouput: cv2 version = 4.5.0

videoCaptureApi = cv2.CAP_ANY # autodetect default API

# this works if conda-forge::opencv=4.5.0 is installed in the local environment
cap = cv2.VideoCapture("/dev/video0", videoCaptureApi)

if not cap.isOpened():
    raise RuntimeError("ERROR! Unable to open camera")

ret, frame1 = cap.read()
prev_img = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    next_img = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_img, next_img, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Flow object contains a vector flow Cartesian information. This vector is pointing at 
    # which direction the flow for each pixels is happening. We basically have x and y coordinates
    # of that vector.
    
    # We want to convert this into polar coordinates to magnitude and angle. We can then turn this
    # into Hue-Saturation-Value mapping. Hue - angle, Saturation - magnitude. If points are moving
    # into particular direction they will coloured the same way. If everything moves left, they'll
    # be coloured red, if everything moves blue, they'll be coloured blue.
    
    magnitude, angle = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)
    hsv_mask[:, :, 0] = angle/2
    hsv_mask[:, :, 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    
    bgr_img = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    cv2.imshow('frame', bgr_img)
    
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break
        
    prev_img = next_img
    
cap.release()
cv2.destroyAllWindows()

cv2 version = 4.5.0


Here are two captures of the video output. 

This one was made when I moved to the right:

![](../img/optical-flow-right-move.png)

This one was made when I moved to the left:

![](../img/optical-flow-left-move.png)